# Homework 4

In [3]:
import pandas as pd
import numpy as np
from sympy import Symbol, lambdify

In [4]:
x1 = Symbol("x1")
x2 = Symbol("x2")

func1 = (5*x1 - x2)**4 + (x1 - 2)**2 + x1 - 2*x2 + 12
func2 = 100*(x2 - x1**2)**2 + (1 - x1)**2 


f1 = lambdify([[x1,x2]], func1, "numpy")
f2 = lambdify([[x1,x2]], func2, "numpy")

gf1 = lambdify([[x1,x2]], func1.diff([[x1, x2]]), "numpy")
gf2 = lambdify([[x1,x2]], func2.diff([[x1, x2]]), "numpy")

grad_f1 = lambda x_arr : np.array(gf1(x_arr)).reshape(1,2)
grad_f2 = lambda x_arr : np.array(gf2(x_arr)).reshape(1,2)

hf1 = lambdify([[x1,x2]], (func1.diff([[x1, x2]])).diff([[x1, x2]]), "numpy")
hf2 = lambdify([[x1,x2]], (func2.diff([[x1, x2]])).diff([[x1, x2]]), "numpy")

hess_f1= lambda x_arr : np.array(hf1(np.array(x_arr).reshape(2,)))
hess_f2= lambda x_arr : np.array(hf2(np.array(x_arr).reshape(2,)))

In [5]:
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

# plot the function
x = np.arange(0,3,0.01)
y = np.arange(0,3,0.01)
X,Y = meshgrid(x, y) # grid of point
Z = f2([X,Y]) # evaluation of the function on the grid

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, 
                      cmap=cm.RdBu,linewidth=0, antialiased=False)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.savefig("graph.png")
plt.show()

<Figure size 640x480 with 2 Axes>

### Useful Functions

In [7]:
np_str = lambda x_k : np.array2string(x_k.reshape(len(x_k)), precision=3, separator=',')

f_str = lambda x : "{0:.4f}".format(x)

In [8]:
class OutputTable:    
    def __init__(self):
        self.table = pd.DataFrame([],columns=['k', 'x^k', 'f(x^k)', 'd^k', 'a^k', 'x^k+1'])
    def add_row(self, k, xk, fxk, dk, ak, xkp):
        self.table.loc[len(self.table)] = [k, np_str(xk), f_str(np.asscalar(fxk)), np_str(dk), ak, np_str(xkp)]
    def print_latex(self):
        print(self.table.to_latex(index=False))

### Exact Line Search

In [11]:
def BisectionMethod(f,epsilon, a=-100,b=100) :
    iteration=0
    while (b - a) >= epsilon:
        x_1 = (a + b) / 2
        fx_1 = f(x_1)
        if f(x_1 + epsilon) <= fx_1:
            a = x_1
        else:
            b = x_1
        iteration+=1
    x_star = (a+b)/2
    return x_star

def ExactLineSearch(f, x0, d, eps=0.0000000001):
    alpha = Symbol('alpha')
    function_alpha = f(np.array(x0)+alpha*np.array(d))
    f_alp = lambdify(alpha, function_alpha, 'numpy')
    alp_star = BisectionMethod(f_alp, epsilon=eps)
    return alp_star

## Steepest Descent Method

In [112]:
def steepestDescentMethod(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    stop = False
    output = OutputTable()
    while(stop == False):
        d = - np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        else:
            a = ExactLineSearch(f,xk,d)
            xkp = xk + a*d
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output

In [113]:
xs1, fs1, outputs1 = steepestDescentMethod(f1, grad_f1, [10,10], 0.001)
xs1, fs1

(array([[ 6.50142886],
        [33.30091895]]), -27.44054872656745)

In [ ]:
print(outputs1.table.to_latex(index=False))

In [ ]:
xs2, fs2, outputs2 = steepestDescentMethod(f1, grad_f1, [-25,-15], 0.001)
xs2, fs2

In [ ]:
print(outputs2.table.to_latex(index=False))

In [ ]:
xs3, fs3, outputs3 = steepestDescentMethod(f2, grad_f2, [2,-4], 0.01)
xs3, fs3

In [ ]:
print(outputs3.table.to_latex(index=False))

In [ ]:
xs4, fs4, outputs4 = steepestDescentMethod(f2, grad_f2, [-2,-3.5], 0.002)
xs4, fs4

In [ ]:
outputs4.table

## Newton's Method

In [12]:
def NewtonsMethod(x_0,epsilon,f,grad_f,Hessian_f):
    xk = np.array(x_0).reshape(2,1)
    k=0
    output = OutputTable()
    while(True):
        d_k=-np.linalg.inv(Hessian_f(xk))@np.transpose(grad_f(xk))
        alpha_k=ExactLineSearch(f,xk,d_k)
        xkp=xk+alpha_k*d_k
        if(np.linalg.norm(grad_f(xk)) < epsilon):
            break
        output.add_row(k, xk, f(xk), d_k, alpha_k, xkp)
        xk = xkp
        k += 1
    output.add_row(k,xk,f(xk),d_k,None,np.array([]))    
    return xk, np.asscalar(f(xk)), output

def NewtonsMethod_vol2(x_0,epsilon,f,grad_f,Hessian_f):
    xk = x_0
    k=0
    H = np.identity(len(x_0))
    output = OutputTable()
    while(True):
        #print(xk)
        a=np.asarray(Hessian_f(xk))
        #print(a)
        #d_k=-np.linalg.inv(a)@np.transpose(np.asarray(grad_f(xk)).reshape(1,2))
        d_k=-np.linalg.inv(a)@np.transpose(np.asarray(gf2(xk)).reshape(1,2))
        alpha_k=ExactLineSearch(f,np.array(xk).reshape(2,1),d_k)
        xkp=[x + y for x, y in zip(x_0,(alpha_k*d_k).flatten().tolist() )]
        #print(xkp)
        print(np.linalg.norm(np.asarray(grad_f(xk)).reshape(1,2)))
        if(np.linalg.norm(np.asarray(grad_f(xk)).reshape(1,2)) < epsilon):
            break
        #output.add_row(k, np.array(xk), f(np.array(xk)), d_k, alpha_k, np.array(xkp))
        xk= xkp
        k += 1
        
    #output.add_row(k,np.array(xk),f(np.array(xk)),d_k,None,np.array([]))    
    return (xk)



In [13]:
x_f1_s1,f1_s1, outputf1_1 = NewtonsMethod([0,1], 0.001,f1,grad_f1,hess_f1)
outputf1_1.table

k              x^k    f(x^k)              d^k        a^k            x^k+1
0    0            [0,1]   15.0000  [ 6.5  ,32.333]   1.000620  [ 6.504,33.353]
1    1  [ 6.504,33.353]  -27.4344  [-0.004,-0.058]   1.048271  [ 6.5  ,33.293]
2    2  [ 6.5  ,33.293]  -27.4406    [0.   ,0.001] -10.547173  [ 6.498,33.283]
3    3  [ 6.498,33.283]  -27.4405    [0.002,0.011]   2.118884  [ 6.503,33.306]
4    4  [ 6.503,33.306]  -27.4405  [-0.003,-0.012]   2.107937  [ 6.497,33.28 ]
5    5  [ 6.497,33.28 ]  -27.4405    [0.003,0.014]   2.598188  [ 6.504,33.315]
6    6  [ 6.504,33.315]  -27.4405  [-0.004,-0.022]   0.390601  [ 6.503,33.307]
7    7  [ 6.503,33.307]  -27.4405  [-0.003,-0.013]   3.190661  [ 6.494,33.265]
8    8  [ 6.494,33.265]  -27.4405    [0.006,0.029]   1.626592  [ 6.504,33.312]
9    9  [ 6.504,33.312]  -27.4405  [-0.004,-0.018]   1.457198  [ 6.498,33.285]
10  10  [ 6.498,33.285]  -27.4405    [0.002,0.008]  10.159400  [ 6.516,33.369]
11  11  [ 6.516,33.369]  -27.4403  [-0.016,-0.076]   1.196289  [ 6.497,33.279]
12  12  [ 6.497,33.279]  -27.4405    [0.003,0.015]   1.275318  [ 6.501,33.298]
13  13  [ 6.501,33.298]  -27.4406  [-0.001,-0.004]  19.678538  [ 6.484,33.217]
14  14  [ 6.484,33.217]  -27.4403    [0.016,0.076]   0.970459  [ 6.5  ,33.291]
15  15  [ 6.5  ,33.291]  -27.4406    [0.   ,0.002]  77.459069  [ 6.536,33.466]
16  16  [ 6.536,33.466]  -27.4392  [-0.036,-0.172]   1.008987  [ 6.5  ,33.292]
17  17  [ 6.5  ,33.292]  -27.4406    [0.   ,0.002]  44.823948  [ 6.514,33.36 ]
18  18  [ 6.514,33.36 ]  -27.4403  [-0.014,-0.066]   1.014407  [ 6.5  ,33.293]
19  19  [ 6.5  ,33.293]  -27.4406    [0.   ,0.001] -24.909975  [ 6.495,33.269]
20  20  [ 6.495,33.269]  -27.4405    [0.005,0.024]   1.586914  [ 6.503,33.308]
21  21  [ 6.503,33.308]  -27.4405  [-0.003,-0.014]   2.103736  [ 6.497,33.278]
22  22  [ 6.497,33.278]  -27.4405    [0.003,0.016]   1.624406  [ 6.502,33.304]
23  23  [ 6.502,33.304]  -27.4405  [-0.002,-0.01 ]   3.624543  [ 6.494,33.268]
24  24  [ 6.494,33.268]  -27.4405    [0.006,0.026]   0.973474  [ 6.5  ,33.293]
25  25  [ 6.5  ,33.293]  -27.4406    [0.   ,0.001] -80.087183  [ 6.488,33.239]
26  26  [ 6.488,33.239]  -27.4403    [0.012,0.055]   0.974348  [ 6.5  ,33.292]
27  27  [ 6.5  ,33.292]  -27.4406    [0.   ,0.001]  35.936538  [ 6.511,33.342]
28  28  [ 6.511,33.342]  -27.4403  [-0.011,-0.048]   1.013706  [ 6.5  ,33.293]
29  29  [ 6.5  ,33.293]  -27.4406    [0.   ,0.001]  53.624399  [ 6.508,33.327]
30  30  [ 6.508,33.327]  -27.4404  [-0.008,-0.033]   1.078259  [ 6.499,33.291]
31  31  [ 6.499,33.291]  -27.4405    [0.001,0.003]   5.138634  [ 6.502,33.304]
32  32  [ 6.502,33.304]  -27.4405  [-0.002,-0.011]   1.676232  [ 6.498,33.287]
33  33  [ 6.498,33.287]  -27.4405    [0.002,0.007]   1.756761  [ 6.501,33.299]
34  34  [ 6.501,33.299]  -27.4405  [-0.001,-0.005]   4.701839  [ 6.495,33.274]
35  35  [ 6.495,33.274]  -27.4405    [0.005,0.02 ]   0.914760  [ 6.5  ,33.292]
36  36  [ 6.5  ,33.292]  -27.4406    [0.   ,0.002]  20.310895  [ 6.508,33.326]
37  37  [ 6.508,33.326]  -27.4403  [-0.008,-0.033]   0.970422  [ 6.5  ,33.295]
38  38  [ 6.5  ,33.295]  -27.4406  [-0.   ,-0.001]  63.990628  [ 6.485,33.23 ]
39  39  [ 6.485,33.23 ]  -27.4401    [0.015,0.064]   1.017001  [ 6.5  ,33.295]
40  40  [ 6.5  ,33.295]  -27.4406  [-0.   ,-0.001]  93.394060  [ 6.477,33.185]
41  41  [ 6.477,33.185]  -27.4399    [0.023,0.109]   1.032173  [ 6.501,33.297]
42  42  [ 6.501,33.297]  -27.4406  [-0.001,-0.004] -12.519098  [ 6.51 ,33.342]
43  43  [ 6.51 ,33.342]  -27.4404  [-0.01 ,-0.048]   0.949097  [ 6.501,33.296]
44  44  [ 6.501,33.296]  -27.4406  [-0.001,-0.002]  37.462615  [ 6.482,33.205]
45  45  [ 6.482,33.205]  -27.4402    [0.018,0.089]   1.029852  [ 6.501,33.296]
46  46  [ 6.501,33.296]  -27.4406  [-0.001,-0.003]  32.451560  [ 6.483,33.21 ]
47  47  [ 6.483,33.21 ]  -27.4402    [0.017,0.084]   0.969642  [ 6.499,33.291]
48  48  [ 6.499,33.291]  -27.4406    [0.001,0.003]  24.890505  [ 6.513,33.354]
49  49  [ 6.513,33.354]  -27.4404  [-0.013,-0.061]   1.

In [51]:
x_f1_s2,f1_s2, outputf1_2 = NewtonsMethod([-25,75], 0.001,f1,grad_f1,hess_f1)
outputf1_2.table

k                x^k           f(x^k)                    d^k        a^k  \
0    0          [-25, 75]  1600000566.0000        [31.5  ,90.833]   2.962919   
1    1  [ 68.332,344.132]        3829.3422    [ -61.832,-309.956]   1.001747   
2    2    [ 6.392,33.634]         -21.7344          [0.108,0.042]   1.756464   
3    3    [ 6.582,33.707]         -27.4338        [-0.082,-0.413]   1.002057   
4    4    [ 6.5  ,33.293]         -27.4406          [0.   ,0.001]  49.983209   
5    5    [ 6.508,33.334]         -27.4405        [-0.008,-0.04 ]   0.732422   
6    6    [ 6.502,33.304]         -27.4405        [-0.002,-0.011]   2.272174   
7    7    [ 6.497,33.28 ]         -27.4405          [0.003,0.014]   4.400717   
8    8      [ 6.51,33.34]         -27.4405        [-0.01 ,-0.047]   0.975800   
9    9    [ 6.5  ,33.295]         -27.4406        [-0.   ,-0.001]  46.081543   
10  10    [ 6.49 ,33.243]         -27.4404          [0.01 ,0.051]   0.975901   
11  11    [ 6.5  ,33.292]         -27.4406          [0.   ,0.001]  99.694440   
12  12    [ 6.525,33.415]         -27.4399        [-0.025,-0.121]   1.003612   
13  13    [ 6.5  ,33.293]         -27.4406  [8.932e-05,4.290e-04]        NaN   

                x^k+1  
0   [ 68.332,344.132]  
1     [ 6.392,33.634]  
2     [ 6.582,33.707]  
3     [ 6.5  ,33.293]  
4     [ 6.508,33.334]  
5     [ 6.502,33.304]  
6     [ 6.497,33.28 ]  
7       [ 6.51,33.34]  
8     [ 6.5  ,33.295]  
9     [ 6.49 ,33.243]  
10    [ 6.5  ,33.292]  
11    [ 6.525,33.415]  
12    [ 6.5  ,33.293]  
13                 []

In [52]:
xs, fs, output = NewtonsMethod([0,7], 1,f2,grad_f2,hess_f2)
output.table

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,7]",4901.0000,"[-7.148e-04,-7.000e+00]",1.000000,"[-7.148e-04, 1.534e-06]"
1,1,"[-7.148e-04, 1.534e-06]",1.0014,"[ 1.001,-0.001]",0.161158,"[ 0.161,-0. ]"
2,2,"[ 0.161,-0. ]",0.7723,"[0.135,0.069]",1.522321,"[0.367,0.106]"
3,3,"[0.367,0.106]",0.4844,"[0.094,0.097]",2.810328,"[0.63 ,0.379]"
4,4,"[0.63 ,0.379]",0.1660,"[0.084,0.123]",2.104425,"[0.807,0.638]"
5,5,"[0.807,0.638]",0.0533,"[0.055,0.101]",3.096049,"[0.976,0.951]"
6,6,"[0.976,0.951]",0.0010,"[0.016,0.034]",NaN,[]


## DFP

In [132]:
def DFP(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    H = np.identity(len(x_0))
    stop = False
    output = OutputTable()
    while(stop == False):
        d = -H @ np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        else:
            a = ExactLineSearch(f,xk,d)
            xkp = xk + a*d
            p = xkp - xk
            q = np.transpose(grad_f(xkp)) - np.transpose(grad_f(xk))
            A = (p @ np.transpose(p)) / (p.transpose() @ q)
            B = - (H @ q @ np.transpose( H @ q)) / (q.transpose() @ H @ q)
            Hkp = H + A + B
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
            H = Hkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output

In [29]:
xs1, fs1, output1 = DFP(f1, grad_f1, [0,0], 0.001)
xs1, fs1

(array([[ 6.50004193],
        [33.29392539]]), -27.44055078634222)

In [30]:
output1.print_latex()

,k,x^k,f(x^k),d^k,a^k,x^k+1
0,0,"[0,0]",16,"[3.,2.]",0.0442505,"[0.133,0.089]"
1,1,"[0.133,0.089]",15.5519,"[0.491,2.493]",13.1149,"[ 6.575,32.79 ]"
2,2,"[ 6.575,32.79 ]",-26.0735,"[-0.057, 0.706]",0.883484,"[ 6.524,33.414]"
3,3,"[ 6.524,33.414]",-27.44,"[-0.024,-0.117]",0.898743,"[ 6.502,33.309]"
4,4,"[ 6.502,33.309]",-27.4405,"[-0.006,-0.038]",0.395203,"[ 6.5 ,33.294]"
5,5,"[ 6.5 ,33.294]",-27.4406,"[-4.193e-05,-2.248e-04]",None,[]


In [ ]:
xs2, fs2, output2 = DFP(f1, grad_f1, [5,5], 0.0001)
xs2, fs2

In [ ]:
output2.print_latex()

In [ ]:
xs3, fs3, output3 = DFP(f2, grad_f2, [1.2,1.6], 1e-9)
xs3, fs3

In [ ]:
output3.print_latex()

In [ ]:
xs4, fs4, output4 = DFP(f2, grad_f2, [-2,-3], 1e-5)
xs4, fs4

In [ ]:
output4.print_latex()

## BFGS

In [84]:
def BFGS(f, grad_f, x_0, epsilon, line_search_tol = 0.0000001):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    H = np.identity(len(x_0))
    stop = False
    output = OutputTable()
    while(stop == False):
        d = -H @ np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        if(k == -1):
            break
        else:
            a = ExactLineSearch(f,xk,d, line_search_tol)
            xkp = xk + a*d
            p = xkp - xk
            q = np.transpose(grad_f(xkp)) - np.transpose(grad_f(xk))
            A = ((1+ np.transpose(q) @ H @ q) / (np.transpose(q) @ p)) * (p @ np.transpose(p)) / (np.transpose(p) @ q)
            B = - (p @ np.transpose(q) @ H + H @ q @ np.transpose(p)) / (np.transpose(q) @ p)
            Hkp = H + A + B
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
            H = Hkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output


In [118]:
xs1, fs1, output1 = BFGS(f1, grad_f1, [0,0], 0.01)
xs1, fs1

(array([[ 6.50060599],
        [33.29678992]]), -27.440550408389115)

In [119]:
output1.print_latex()

\begin{tabular}{llllrl}
\toprule
 k &              x\textasciicircum k &    f(x\textasciicircum k) &              d\textasciicircum k &        a\textasciicircum k &            x\textasciicircum k+1 \\
\midrule
 0 &            [0,0] &   16.0000 &          [3.,2.] &   0.047375 &    [0.142,0.095] \\
 1 &    [0.142,0.095] &   15.5482 &    [0.914,4.848] &   6.308132 &  [ 5.908,30.676] \\
 2 &  [ 5.908,30.676] &  -26.4980 &    [2.239,7.413] &   0.106881 &  [ 6.147,31.468] \\
 3 &  [ 6.147,31.468] &  -27.3030 &    [0.005,0.025] &  72.460937 &  [ 6.508,33.314] \\
 4 &  [ 6.508,33.314] &  -27.4391 &  [-0.003,-0.007] &   2.636336 &  [ 6.501,33.297] \\
 5 &  [ 6.501,33.297] &  -27.4406 &  [-0.003,-0.007] &        NaN &               [] \\
\bottomrule
\end{tabular}



In [87]:
xs2, fs2, output2 = BFGS(f1, grad_f1, [10,10], 0.01)
xs2, fs2

(array([[ 6.50006022],
        [32.48387107]]), -26.217139871720036)

In [120]:
output2.print_latex()

\begin{tabular}{llllrl}
\toprule
 k &              x\textasciicircum k &        f(x\textasciicircum k) &                      d\textasciicircum k &        a\textasciicircum k &            x\textasciicircum k+1 \\
\midrule
 0 &          [10,10] &  2560066.0000 &    [-1280017.,  256002.] &   0.000006 &  [ 2.311,11.538] \\
 1 &  [ 2.311,11.538] &       -8.6681 &            [0.322,1.611] &  13.000301 &  [ 6.5  ,32.484] \\
 2 &  [ 6.5  ,32.484] &      -26.2171 &  [-2.837e-09,-1.415e-08] & -17.036686 &  [ 6.5  ,32.484] \\
 3 &  [ 6.5  ,32.484] &      -26.2171 &  [-2.837e-09,-1.415e-08] &        NaN &               [] \\
\bottomrule
\end{tabular}



In [121]:
xs3, fs3, output3 = BFGS(f2, grad_f2, [0,0], 0.01)
xs3, fs3

(array([[1.08132106],
        [1.16412038]]), 0.009249780378701711)

In [122]:
output3.print_latex()

\begin{tabular}{llllrl}
\toprule
 k &            x\textasciicircum k &  f(x\textasciicircum k) &              d\textasciicircum k &        a\textasciicircum k &          x\textasciicircum k+1 \\
\midrule
 0 &          [0,0] &  1.0000 &          [2.,0.] &   0.080631 &  [0.161,0.   ] \\
 1 &  [0.161,0.   ] &  0.7711 &  [13.526, 5.201] &   0.009728 &  [0.293,0.051] \\
 2 &  [0.293,0.051] &  0.6237 &    [0.238,0.351] &   3.563826 &  [1.141,1.303] \\
 3 &  [1.141,1.303] &  0.0201 &    [0.1  ,0.044] &   0.005316 &  [1.141,1.303] \\
 4 &  [1.141,1.303] &  0.0200 &  [-0.001,-0.003] &  40.222919 &  [1.081,1.164] \\
 5 &  [1.081,1.164] &  0.0092 &  [-0.001,-0.003] &        NaN &             [] \\
\bottomrule
\end{tabular}



In [126]:
xs4, fs4, output4 = BFGS(f2, grad_f2, [10,10], 0.001, line_search_tol=10**(-9))
xs4, fs4

(array([[ 3.21490915],
        [10.33410792]]), 4.906057543360549)

In [127]:
output4.print_latex()

\begin{tabular}{llllrl}
\toprule
 k &              x\textasciicircum k &       f(x\textasciicircum k) &                    d\textasciicircum k &        a\textasciicircum k &            x\textasciicircum k+1 \\
\midrule
 0 &          [10,10] &  810081.0000 &    [-360018.,  18000.] &   0.000019 &  [ 3.215,10.339] \\
 1 &  [ 3.215,10.339] &       4.9073 &        [-0.024,-0.483] &   0.010887 &  [ 3.215,10.334] \\
 2 &  [ 3.215,10.334] &       4.9061 &  [8.995e-08,1.800e-06] &  81.190871 &  [ 3.215,10.334] \\
 3 &  [ 3.215,10.334] &       4.9061 &  [8.995e-08,1.800e-06] &        NaN &               [] \\
\bottomrule
\end{tabular}

